# TRAINING I3W


# A) Create Envorinment, Vehicles etc

### General Parameter

In [1]:
# Define horizon as a variable to ensure consistent use across notebook (length of one rollout)
HORIZON=500

# name of the experiment
experiment_name = "IntersectionExample"

# scenario class
import flow.scenarios as scenarios
print("Available scenarios:")
print(scenarios.__all__)
scenario_name = "IntersectionTWScenario"

# environment class
import flow.multiagent_envs as flowenvs
print("\nAvailable environments:")
print(flowenvs.__all__)
env_name = "MultiAgentTeamSpiritIntersectionEnv"

Available scenarios:
['Scenario', 'BayBridgeScenario', 'BayBridgeTollScenario', 'BottleneckScenario', 'Figure8Scenario', 'SimpleGridScenario', 'HighwayScenario', 'LoopScenario', 'MergeScenario', 'TwoLoopsOneMergingScenario', 'MultiLoopScenario', 'IntersectionScenario', 'IntersectionScenarioTW']

Available environments:
['MultiEnv', 'MultiAgentAccelEnv', 'MultiWaveAttenuationPOEnv', 'MultiAgentIntersectionEnv', 'MultiAgentTeamSpiritIntersectionEnv']


### Net Parameter

In [2]:
from flow.core.params import NetParams
from flow.scenarios.intersection import ADDITIONAL_NET_PARAMS

additionalNetParams = {
            "edge_length": 40,
            "lanes": 1,
            "speed_limit": 30
        }

net_params = NetParams( no_internal_links=False,                  #default: True   !! damit Kreuzungen nicht überspr. werden
                        inflows=None,                             #default: None
                        osm_path=None,                            #default: None
                        netfile=None,                             #default: None
                        additional_params=additionalNetParams     #default: None   !!
                      )

### InitialConfig Parameter

In [3]:
from flow.core.params import InitialConfig

initial_config = InitialConfig( shuffle=True,                            #default: False         !!
                                spacing="custom",                        #default: "uniform"     !!
                                min_gap=10,                              #default: 0
                                perturbation=39.99,                      #default: 0.0            !!        
                                x0=0,                                    #default: 0
                                bunching=0,                              #default: 0
                                lanes_distribution=float("inf"),         #default: float("inf")
                                edges_distribution="all",                #default: "all"
                                additional_params=None )                 #default: None

### SUMO Parameter

In [4]:
from flow.core.params import SumoParams

sumo_params = SumoParams( port = None,                  #default: None
                          sim_step=0.1,                 #default: 0.1
                          emission_path=None,           #default: None
                          lateral_resolution=None,      #default: None
                          no_step_log=True,             #default: True
                          render=False,                 #default: False
                          save_render=False,            #default: False
                          sight_radius=25,              #default: 25
                          show_radius=False,            #default: False
                          pxpm=2,                       #default: 2
                          overtake_right=False,         #default: False    
                          seed=None,                    #default: None
                          restart_instance=False,       #default: False
                          print_warnings=True,          #default: True
                          teleport_time=-1,             #default: -1
                          num_clients=1,                #default: 1
                          sumo_binary=None )            #default: None

### Environment Parameter

In [5]:
from flow.core.params import EnvParams

additionalEnvParams = {
        # maximum acceleration of autonomous vehicles
        "max_accel": 3,
        # maximum deceleration of autonomous vehicles
        "max_decel": 3,
        "target_velocity": 30
    }

env_params = EnvParams( additional_params=additionalEnvParams, #default: None    !!
                        horizon=HORIZON,                       #default: 500     !!
                        warmup_steps=0,                        #default: 0       
                        sims_per_step=1,                       #default: 1
                        evaluate=False )                       #default: False

### Vehicles Parameter

In [6]:
from flow.core.params import VehicleParams

# import vehicles dynamics models
#from flow.controllers import SumoCarFollowingController
from flow.controllers import ContinuousRouter
#from flow.controllers.lane_change_controllers import SumoLaneChangeController
from flow.controllers.lane_change_controllers import StaticLaneChanger
from flow.controllers import RLController
from flow.core.params import SumoLaneChangeParams
from flow.core.params import SumoCarFollowingParams
from random import *

vehicles = VehicleParams()

#### Add RL-Agent controlled vehicles 

In [7]:
# car following parameters, default: None
cf_parameter = SumoCarFollowingParams(
                speed_mode="aggressive")
# lane change parameters, default: None
lc_parameter =  None

vehicles.add( # name of the vehicle
                veh_id = "rl",
              # acceleration controller, default: (SumoCarFollowingController, {})
                acceleration_controller=(RLController, {}),
              # lane_change_controller, default: (SumoLaneChangeController, {})
                lane_change_controller=(StaticLaneChanger,{}),
              # routing controller, default: None
                routing_controller=(ContinuousRouter, {}),
              # initial speed, default: 0
                initial_speed=0,
              # number of vehicles, default: 1 
                num_vehicles=2,
                
                car_following_params=cf_parameter
              # speed mode, default: "right_of_way"
                #speed_mode="aggressive",
              # lane change mode, default: "no_lat_collide"
                #lane_change_mode="aggressive", 
              # car following parameter, default: None
                #sumo_car_following_params=cf_parameter,
              # lane change parameter, default: None
                #sumo_lc_params=lc_parameter
)

### Flow Parameter

In [8]:
# Creating flow_params. Make sure the dictionary keys are as specified. 
flow_params = dict( # name of the experiment
                      exp_tag=experiment_name,
                    # name of the flow environment the experiment is running on
                      env_name=env_name,
                    # name of the scenario class the experiment uses
                      scenario=scenario_name,
                    # simulator that is used by the experiment
                      simulator='traci',
                    # sumo-related parameters (see flow.core.params.SumoParams)
                      sim=sumo_params,
                    # environment related parameters (see flow.core.params.EnvParams)
                      env=env_params,
                    # network-related parameters (see flow.core.params.NetParams and
                    # the scenario's documentation or ADDITIONAL_NET_PARAMS component)
                      net=net_params,
                    # vehicles to be placed in the network at the start of a rollout 
                    # (see flow.core.vehicles.Vehicles)
                      veh=vehicles,
                   # (optional) parameters affecting the positioning of vehicles upon 
                   # initialization/reset (see flow.core.params.InitialConfig)
                      initial=initial_config
                )

# B) Training

In [9]:
import json

import ray
try:
    from ray.rllib.agents.agent import get_agent_class
except ImportError:
    from ray.rllib.agents.registry import get_agent_class
from ray.tune import run_experiments
from ray.tune.registry import register_env

from flow.utils.registry import make_create_env
from flow.utils.rllib import FlowParamsEncoder

from ray import tune
from ray.rllib.agents.ppo.ppo_policy_graph import PPOPolicyGraph

In [10]:
# number of parallel workers
N_CPUS = 2
# number of rollouts per training iteration
N_ROLLOUTS = 20

ray.init(redirect_output=True, num_cpus=N_CPUS+1)

Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-02-23_01-09-09_12703/logs.
Waiting for redis server at 127.0.0.1:21994 to respond...
Waiting for redis server at 127.0.0.1:49613 to respond...
Starting the Plasma object store with 6.554658406 GB memory using /dev/shm.

View the web UI at http://localhost:8889/notebooks/ray_ui.ipynb?token=e101b9a8d3f9ba6e6f712de31e0b9c203bc003a504a165d1



{'node_ip_address': '192.168.2.102',
 'object_store_addresses': ['/tmp/ray/session_2019-02-23_01-09-09_12703/sockets/plasma_store'],
 'raylet_socket_names': ['/tmp/ray/session_2019-02-23_01-09-09_12703/sockets/raylet'],
 'redis_address': '192.168.2.102:21994',
 'webui_url': 'http://localhost:8889/notebooks/ray_ui.ipynb?token=e101b9a8d3f9ba6e6f712de31e0b9c203bc003a504a165d1'}

In [11]:
# The algorithm or model to train. This may refer to "
#      "the name of a built-on algorithm (e.g. RLLib's DQN "
#      "or PPO), or a user-defined trainable function or "
#      "class registered in the tune registry.")
alg_run = "PPO"

agent_cls = get_agent_class(alg_run)
config = agent_cls._default_config.copy()
config["num_workers"] = N_CPUS  # number of parallel workers
config["train_batch_size"] = HORIZON * N_ROLLOUTS  # batch size
config["gamma"] = 0.999  # discount rate
config["model"].update({"fcnet_hiddens": [64, 32]})  # size of hidden layers in network
config["use_gae"] = True  # using generalized advantage estimation
config["lambda"] = 0.97  
#config["sgd_minibatch_size"] = min(16 * 1024, config["train_batch_size"])  # stochastic gradient descent
#config["sample_batch_size"] = config["train_batch_size"]/config["num_workers"] # 200 default, trotzdem zu hoch?
config["kl_target"] = 0.02  # target KL divergence
config["num_sgd_iter"] = 10  # number of SGD iterations
config["horizon"] = HORIZON  # rollout horizon

# save the flow params for replay
flow_json = json.dumps(flow_params, cls=FlowParamsEncoder, sort_keys=True,
                       indent=4)  # generating a string version of flow_params
config['env_config']['flow_params'] = flow_json  # adding the flow_params to config dict
config['env_config']['run'] = alg_run

# Call the utility function make_create_env to be able to 
# register the Flow env for this experiment
create_env, gym_name = make_create_env(params=flow_params, version=0)

# Register as rllib env with Gym
register_env(gym_name, create_env)

In [12]:
# multi agent policy mapping
test_env = create_env()
obs_space = test_env.observation_space
act_space = test_env.action_space

def gen_policy():
    return (PPOPolicyGraph, obs_space, act_space, {})

# Setup PG with an ensemble of `num_policies` different policy graphs
policy_graphs = {'rl_0': gen_policy(), 'rl_1': gen_policy()}
    
def policy_mapping_fn(agent_id):
    return agent_id

config.update({
        'multiagent': {
            'policy_graphs': policy_graphs,
            'policy_mapping_fn': tune.function(policy_mapping_fn)
        }
    })

 Starting SUMO on port 48217


1.6038076979708429
11.582291891152966


In [13]:
trials = run_experiments({
    flow_params["exp_tag"]: {
        "run": alg_run,  # RL algorithm to run
        "env": gym_name,  # environment name generated earlier
        "config": {  # configuration params (must match "run" value)
            **config
        },
        "checkpoint_freq": 1,  # number of iterations between checkpoints
        "max_failures": 999,
        "stop": {  # stopping conditions
            "training_iteration": 1000,  # number of iterations to stop after
        },
    },
})

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/3 CPUs, 0/1 GPUs
Memory usage on this node: 3.2/16.4 GB

Created LogSyncer for /home/thorsten/ray_results/IntersectionExample/PPO_MultiAgentTeamSpiritIntersectionEnv-v0_0_2019-02-23_01-09-11i4dg0a94 -> 
== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 3.2/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentTeamSpiritIntersectionEnv-v0_0:	RUNNING

Result for PPO_MultiAgentTeamSpiritIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-02-23_01-10-07
  done: false
  episode_len_mean: 400.1666666666667
  episode_reward_max: 145.38326111257012
  episode_reward_mean: -8.759566947312722
  episode_reward_min: -181.94378557141593
  episodes_this_iter: 24
  episodes_total: 24
  experiment_id: 1e6021831b8d437c9eb317160a4e5781
  hostname: Gandalf
  info:
    grad_time_ms: 4278.545
    load_time_ms: 143.977
    num

Result for PPO_MultiAgentTeamSpiritIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-02-23_01-11-41
  done: false
  episode_len_mean: 398.7
  episode_reward_max: 207.50015774012948
  episode_reward_mean: 25.34166855334402
  episode_reward_min: -176.02688496905031
  episodes_this_iter: 26
  episodes_total: 124
  experiment_id: 1e6021831b8d437c9eb317160a4e5781
  hostname: Gandalf
  info:
    grad_time_ms: 3472.252
    load_time_ms: 30.896
    num_steps_sampled: 50000
    num_steps_trained: 50000
    rl_0:
      cur_kl_coeff: 0.012500000186264515
      cur_lr: 4.999999873689376e-05
      entropy: 1.4442048072814941
      kl: 0.005586014594882727
      policy_loss: -0.00443550618365407
      total_loss: 151.82093811035156
      vf_explained_var: 0.2782227396965027
      vf_loss: 151.82530212402344
    rl_1:
      cur_kl_coeff: 0.012500000186264515
      cur_lr: 4.999999873689376e-05
      entropy: 1.414603352546692
      kl: 0.0022687746677547693
      policy_loss: -0.002418189542368

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 4.1/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentTeamSpiritIntersectionEnv-v0_0:	RUNNING [pid=12746], 214 s, 9 iter, 90000 ts, 48.7 rew

Result for PPO_MultiAgentTeamSpiritIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-02-23_01-13-39
  done: false
  episode_len_mean: 333.85
  episode_reward_max: 208.21025410316182
  episode_reward_mean: 38.5822240009209
  episode_reward_min: -179.29151748662593
  episodes_this_iter: 33
  episodes_total: 269
  experiment_id: 1e6021831b8d437c9eb317160a4e5781
  hostname: Gandalf
  info:
    grad_time_ms: 3369.236
    load_time_ms: 16.579
    num_steps_sampled: 100000
    num_steps_trained: 100000
    rl_0:
      cur_kl_coeff: 0.0003906250058207661
      cur_lr: 4.999999873689376e-05
      entropy: 1.439549207687378
      kl: 0.004999891854822636
      policy_loss: -0.00337318773

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 3.9/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentTeamSpiritIntersectionEnv-v0_0:	RUNNING [pid=12746], 333 s, 14 iter, 140000 ts, 7.81 rew

Result for PPO_MultiAgentTeamSpiritIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-02-23_01-15-38
  done: false
  episode_len_mean: 305.45
  episode_reward_max: 203.06118103919619
  episode_reward_mean: 25.038074221456988
  episode_reward_min: -183.48026746832272
  episodes_this_iter: 28
  episodes_total: 431
  experiment_id: 1e6021831b8d437c9eb317160a4e5781
  hostname: Gandalf
  info:
    grad_time_ms: 3271.985
    load_time_ms: 2.419
    num_steps_sampled: 150000
    num_steps_trained: 150000
    rl_0:
      cur_kl_coeff: 1.220703143189894e-05
      cur_lr: 4.999999873689376e-05
      entropy: 1.4099640846252441
      kl: 0.0023611080832779408
      policy_loss: -0.001374

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 3.9/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentTeamSpiritIntersectionEnv-v0_0:	RUNNING [pid=12746], 450 s, 19 iter, 190000 ts, 61 rew

Result for PPO_MultiAgentTeamSpiritIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-02-23_01-17-35
  done: false
  episode_len_mean: 254.06
  episode_reward_max: 211.2370153478997
  episode_reward_mean: 49.256519552869385
  episode_reward_min: -177.89501972044874
  episodes_this_iter: 42
  episodes_total: 608
  experiment_id: 1e6021831b8d437c9eb317160a4e5781
  hostname: Gandalf
  info:
    grad_time_ms: 3263.241
    load_time_ms: 2.497
    num_steps_sampled: 200000
    num_steps_trained: 200000
    rl_0:
      cur_kl_coeff: 3.814697322468419e-07
      cur_lr: 4.999999873689376e-05
      entropy: 1.4443562030792236
      kl: 0.003174273297190666
      policy_loss: -0.0019141527

...

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 7.4/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentTeamSpiritIntersectionEnv-v0_0:	RUNNING [pid=12746], 22667 s, 979 iter, 9790000 ts, 161 rew

Result for PPO_MultiAgentTeamSpiritIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-02-23_07-28-35
  done: false
  episode_len_mean: 111.69
  episode_reward_max: 230.3789336521863
  episode_reward_mean: 158.02653691627953
  episode_reward_min: -163.72144639998803
  episodes_this_iter: 90
  episodes_total: 85078
  experiment_id: 1e6021831b8d437c9eb317160a4e5781
  hostname: Gandalf
  info:
    grad_time_ms: 3099.294
    load_time_ms: 2.531
    num_steps_sampled: 9800000
    num_steps_trained: 9800000
    rl_0:
      cur_kl_coeff: 0.0
      cur_lr: 4.999999873689376e-05
      entropy: -0.7018633484840393
      kl: 0.026223478838801384
      policy_loss: 0.003589326748624444


== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 7.4/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentTeamSpiritIntersectionEnv-v0_0:	RUNNING [pid=12746], 22782 s, 984 iter, 9840000 ts, 147 rew

Result for PPO_MultiAgentTeamSpiritIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-02-23_07-30-31
  done: false
  episode_len_mean: 116.95
  episode_reward_max: 232.69858266714823
  episode_reward_mean: 162.65559225303076
  episode_reward_min: -168.64136532521718
  episodes_this_iter: 85
  episodes_total: 85511
  experiment_id: 1e6021831b8d437c9eb317160a4e5781
  hostname: Gandalf
  info:
    grad_time_ms: 3086.345
    load_time_ms: 2.555
    num_steps_sampled: 9850000
    num_steps_trained: 9850000
    rl_0:
      cur_kl_coeff: 0.0
      cur_lr: 4.999999873689376e-05
      entropy: -0.6592262983322144
      kl: 0.016444748267531395
      policy_loss: 0.001776738441549241

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 7.5/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentTeamSpiritIntersectionEnv-v0_0:	RUNNING [pid=12746], 22897 s, 989 iter, 9890000 ts, 159 rew

Result for PPO_MultiAgentTeamSpiritIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-02-23_07-32-26
  done: false
  episode_len_mean: 116.24
  episode_reward_max: 234.02537217439811
  episode_reward_mean: 150.42853140848217
  episode_reward_min: -169.8375626692768
  episodes_this_iter: 85
  episodes_total: 85950
  experiment_id: 1e6021831b8d437c9eb317160a4e5781
  hostname: Gandalf
  info:
    grad_time_ms: 3073.85
    load_time_ms: 2.526
    num_steps_sampled: 9900000
    num_steps_trained: 9900000
    rl_0:
      cur_kl_coeff: 0.0
      cur_lr: 4.999999873689376e-05
      entropy: -0.5643344521522522
      kl: 0.021824868395924568
      policy_loss: 0.00533814774826169
  

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 7.3/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentTeamSpiritIntersectionEnv-v0_0:	RUNNING [pid=12746], 23012 s, 994 iter, 9940000 ts, 156 rew

Result for PPO_MultiAgentTeamSpiritIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-02-23_07-34-21
  done: false
  episode_len_mean: 111.0
  episode_reward_max: 234.4029011180867
  episode_reward_mean: 146.53507325442615
  episode_reward_min: -174.15894386203524
  episodes_this_iter: 89
  episodes_total: 86382
  experiment_id: 1e6021831b8d437c9eb317160a4e5781
  hostname: Gandalf
  info:
    grad_time_ms: 3052.295
    load_time_ms: 2.499
    num_steps_sampled: 9950000
    num_steps_trained: 9950000
    rl_0:
      cur_kl_coeff: 0.0
      cur_lr: 4.999999873689376e-05
      entropy: -0.6908711791038513
      kl: 0.019373439252376556
      policy_loss: 0.0013647997984662652


== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 7.3/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentTeamSpiritIntersectionEnv-v0_0:	RUNNING [pid=12746], 23128 s, 999 iter, 9990000 ts, 168 rew

Result for PPO_MultiAgentTeamSpiritIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-02-23_07-36-17
  done: true
  episode_len_mean: 110.81
  episode_reward_max: 237.48101045400855
  episode_reward_mean: 157.08994961003972
  episode_reward_min: -178.83785417867946
  episodes_this_iter: 89
  episodes_total: 86829
  experiment_id: 1e6021831b8d437c9eb317160a4e5781
  hostname: Gandalf
  info:
    grad_time_ms: 3109.352
    load_time_ms: 2.3
    num_steps_sampled: 10000000
    num_steps_trained: 10000000
    rl_0:
      cur_kl_coeff: 0.0
      cur_lr: 4.999999873689376e-05
      entropy: -0.7738745212554932
      kl: 0.02327972836792469
      policy_loss: 0.0015150641556829214
